In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "llama2"


In [4]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings

model = OllamaLLM(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

response = model.invoke("Tell me a joke")

print(response)


Sure, here's one:

Why don't scientists trust atoms?
Because they make up everything!

I hope you found that amusing! Do you want to hear another one?


In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

print(response)


Sure, here's one:

Why don't scientists trust atoms?
Because they make up everything!

I hope you found that amusing! Do you want to hear another one?


In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [7]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer would be:\n\n"My name is Santiago."'

In [15]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("test.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'test.pdf', 'page': 0}, page_content='Estrategias de RAG y evaluaci´ on de preguntas en modelos de\nlenguaje\nFelipe Clari´ a, Santiago Fada\nFAMaFyC, UNC, Argentina\nResumen\nEl uso de Retrieval Augmented Generation (RAG) ha ganado popularidad como una t´ ecnica\npara mejorar la generaci´ on de respuestas en modelos de lenguaje natural. Esta metodolog´ ıa permite\nintegrar informaci´ on externa al modelo con el objetivo de proporcionar contexto a las respuestas ge-\nneradas. En este estudio, exploramos c´ omo RAG puede optimizar la respuesta a preguntas complejas\ny especializadas, mejorando la calidad de las interacciones con los modelos de lenguaje.\nIntroducci´ on\nLos Large Language Model (LLM) han alcanzado un ´ exito notable en diversas tareas de procesamiento\nde texto, pero todav´ ıa enfrentan limitaciones en tareas espec´ ıficas de dominio o que requieren un alto\nnivel de conocimiento [1]. Estos modelos son propensos a producir “alucinaciones” c

In [17]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [18]:
retriever = vectorstore.as_retriever()
retriever.invoke("semana 5")

[Document(metadata={'source': 'test.pdf', 'page': 1}, page_content='Preguntas de contexto cerrado Resueltas dentro del conocimiento del modelo.\nPreguntas contextuales Requieren informaci´ on adicional no contenida en el modelo.\nPreguntas ambiguas o complejas No pueden ser resueltas adecuadamente con o sin informaci´ on\nadicional debido a la falta de datos precisos.\nPreguntas especializadas Implican la necesidad de informaci´ on filtrada o criterios espec´ ıficos.\nPlanificaci´ on futura\nSemana 1: Revisi´ on de literatura sobre RAG y definici´ on de contextos espec´ ıficos. Especificaci´ on\ny formulaci´ on de preguntas.\nSemana 2: Extracci´ on y preparaci´ on de datos de fuentes relevantes. Implementaci´ on de t´ ecnicas\nde preprocesamiento y limpieza de datos.\nSemana 3: Desarrollo y entrenamiento de modelos utilizando los datos extra´ ıdos. Evaluaci´ on inicial\nde la calidad de las respuestas generadas mediante preguntas de contexto cerrado y contextuales.\nSemana 4: Experimen

In [19]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "¿De qué trata el documento?",
    "¿Cuál es la hipótesis?",
    "¿Cuáles son los objetivos de cada semana?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: ¿De qué trata el documento?


In [ ]:
chain.batch([{"question": q} for q in questions])

In [ ]:
for s in chain.stream({"question": "¿Qué es un sistema de amortización?"}):
    print(s, end="", flush=True)